# MIDS W207 Fall 2017 Final Project¶
## Data Set Up - Data Cleaning and Feature Engineering
Laura Williams, Kim Vignola, Cyprian Gascoigne  
SF Crime Classification

This notebook reads raw data (saved in a zip file) from Kaggle, processes and organizes the data for training a variety of machine learning models, and outputs the data as zipped csv files that other notebooks can unzip and use to train different models.

The intention is that data cleaning and/or feature engineering will be added to this file as we progress through the project and look for additional way to process the data to improve our predictions.

For ease of processing this data, exploratory data analysis will be done separately.

Resulting zipped files will include: 

1) train_data.csv and train_labels.csv - includes 80% of the total training data, for training models that are not yet going to be submitted to Kaggle

2) dev_data.csv and dev_labels.csv - includes 20% of the total training data, for testing models before they are submitted to Kaggle

3) train_data_all.csv and train_labels_all.csv - includes all the training data. After testing models with the train and dev data split above, train the model from this full set of data for submission to Kaggle.

4) test_data_all.csv - create predictions on this data for submission to Kaggle.

Weather data for San Franscisco County was added to this analysis.
Source: https://www.ncdc.noaa.gov/cdo-web/search
Report: Daily Summaries, Date Range: 1/1/2003 - 12/31/2015, Search for: Counties/San Francisco, Station: SAN FRANCISCO DOWNTOWN, CA US, Metrics: Precipitation, Maximum Temperature, Minimum Temperature.


#### NOTE: holidays package is not native with anaconda and may need to be installed

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import zipfile
from datetime import datetime, timedelta, date
import holidays
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
# Unzip raw data into a subdirectory 
unzip_files = zipfile.ZipFile("raw_data.zip", "r")
unzip_files.extractall("raw_data")
unzip_files.close()

In [3]:
# Read CSV files into pandas dataframes
train = pd.read_csv("raw_data/train.csv")
test = pd.read_csv("raw_data/test.csv")
weather = pd.read_csv("raw_data/SF_county.csv")



In [4]:
# extract month, year and hour from both datasets
train["month"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").month)
train["year"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").year)
train["hour"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").hour)
train["day"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").day)

test["month"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").month)
test["year"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").year)
test["hour"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").hour)
test["day"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").day)

# map holidays
US_Holidays = holidays.UnitedStates()
train["holidays"] = train["Dates"].map(lambda x: x in US_Holidays)
test["holidays"] = test["Dates"].map(lambda x: x in US_Holidays)


In [5]:
#I Will mess around with this later
#print("2015-1-1" in US_Holidays)
#print(date(2015,1,1)in US_Holidays)
#print(date(train["year"], train["month"], 1) + timedelta(days = 1))

In [6]:
# Pull out first day of the month as it ranks first for crime volume and specific crimes may be associated with the day
train["first_day"] = [1 if x==1 else 0 for x in train["day"]]
test["first_day"] = [1 if x==1 else 0 for x in test["day"]]

In [7]:
# create a bucket variable for month_year

train["month_year"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
train["month_year"] = train["month_year"].map(lambda x: datetime.strftime(x,"%Y-%m"))

test["month_year"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
test["month_year"] = test["month_year"].map(lambda x: datetime.strftime(x,"%Y-%m"))

# would month_day have any value?
#train["month_day"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
#train["month_day"] = train["month_day"].map(lambda x: datetime.strftime(x,"%m-%d"))
#test["month_day"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
#test["month_day"] = test["month_day"].map(lambda x: datetime.strftime(x,"%m-%d"))

In [8]:
# parse out day of year for bucketing seasons

train["doy"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timetuple().tm_yday)
test["doy"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timetuple().tm_yday)

train["spring"] = [1 if x in range(81,173) else 0 for x in train["doy"]]
train["summer"] = [1 if x in range(173,265) else 0 for x in train["doy"]]
train["fall"] = [1 if x in range(265,356) else 0 for x in train["doy"]]
train["winter"] = [1 if x in range(1,81) or x in range(356,366) else 0 for x in train["doy"]]

test["spring"] = [1 if x in range(81,173) else 0 for x in test["doy"]]
test["summer"] = [1 if x in range(173,265) else 0 for x in test["doy"]]
test["fall"] = [1 if x in range(265,356) else 0 for x in test["doy"]]
test["winter"] = [1 if x in range(1,81) or x in range(356,366) else 0 for x in test["doy"]]

In [9]:
# create a dictionary for bucketing hours
time_periods = {6:"early_morning", 7:"early_morning", 8:"early_morning", 
               9:"late_morning", 10:"late_morning", 11:"late_morning",
              12:"early_afternoon", 13:"early_afternoon", 14:"early_afternoon",
              15:"late_afternoon", 16:"late_afternoon", 17:"late_afternoon",
              18:"early_evening",  19:"early_evening",  20:"early_evening",
              21:"late_evening", 22:"late_evening", 23:"late_evening",
              0:"late_night", 1:"late_night", 2:"late_night",
              3:"late_night", 4:"late_night", 5:"late_night"}

# map time periods to dayparts
train["dayparts"] = train["hour"].map(time_periods)
test["dayparts"] = test["hour"].map(time_periods)

In [10]:
# clean up weather data
del weather['NAME']
weather["SNOW"] = weather["SNOW"].fillna(0)

In [11]:
# drop time from train and test date fields to be able to map Dates against weather data; remove hyphens too.
train["Dates"] = train["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
train["Dates"] = train["Dates"].map(lambda x: datetime.strftime(x,"%Y%m%d"))
test["Dates"] = test["Dates"].map(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
test["Dates"] = test["Dates"].map(lambda x: datetime.strftime(x,"%Y%m%d"))

# Convert Weather DATE to same format as train and test data
weather["DATE"] = weather["DATE"].map(lambda x: datetime.strptime(x,"%m/%d/%y"))
weather["DATE"] = weather["DATE"].map(lambda x: datetime.strftime(x,"%Y%m%d"))

In [12]:
# convert date objects to numeric
train["Dates"] = pd.to_numeric(train["Dates"])
test["Dates"] = pd.to_numeric(test["Dates"])
weather["DATE"] = pd.to_numeric(weather["DATE"])
print(type(train["Dates"][0]))

<class 'numpy.int64'>


In [13]:
# left merge weather data based on dates
weather_train = pd.merge(train, weather, how='left', left_on="Dates", right_on = "DATE")
del weather_train['DATE']
del weather_train["SNOW"]
weather_test = pd.merge(test, weather, how='left', left_on="Dates", right_on = "DATE")
del weather_test['DATE']
del weather_test["SNOW"]

Next, fix outliers

In [14]:
# Data indicates outliers with latitude = 90. Test data has these same outliers.
# Set latitiude to the median of the district where the crime occured.
districts = set(weather_train["PdDistrict"])
medians = {el:0 for el in districts}
for district in districts:
    medians[district] = weather_train["Y"][weather_train["PdDistrict"] == district].median()
weather_train.loc[weather_train.Y > 38, "Y"] = weather_train[weather_train.Y > 38]["PdDistrict"].map(lambda x: 
                                                                                                     medians[x])
weather_test.loc[weather_test.Y > 38, "Y"] = weather_test[weather_test.Y > 38]["PdDistrict"].map(lambda x : 
                                                                                                 medians[x])

In [15]:
#print new shape
print(weather_train.shape)
print(weather_test.shape)

print("Cases removed from train data =", np.sum(878049 - weather_train.shape[0]))
print("Cases removed from test data =", np.sum(884262 - weather_test.shape[0]))
print("Cases fixed in the train data =", len(train[train.Y>38]))
print("Cases fixed in the test data =", len(test[test.Y > 38]))

(878049, 25)
(884262, 23)
Cases removed from train data = 0
Cases removed from test data = 0
Cases fixed in the train data = 67
Cases fixed in the test data = 76


In [16]:
print(train.columns)
print(weather_train.columns)
print(test.columns)
print(weather_test.columns)

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'month', 'year', 'hour', 'day',
       'holidays', 'first_day', 'month_year', 'doy', 'spring', 'summer',
       'fall', 'winter', 'dayparts'],
      dtype='object')
Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'month', 'year', 'hour', 'day',
       'holidays', 'first_day', 'month_year', 'doy', 'spring', 'summer',
       'fall', 'winter', 'dayparts', 'PRCP', 'TMAX', 'TMIN'],
      dtype='object')
Index(['Id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'month',
       'year', 'hour', 'day', 'holidays', 'first_day', 'month_year', 'doy',
       'spring', 'summer', 'fall', 'winter', 'dayparts'],
      dtype='object')
Index(['Id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'month',
       'year', 'hour', 'day', 'holidays', 'first_day', 'month_year', 'doy',
       'spring', 'summer', 'fall', 'wi

In [17]:
weather_train["holidays"] = weather_train["holidays"].astype(int)
weather_test["holidays"] = weather_test["holidays"].astype(int)
print(type(weather_train["holidays"][0]))
print(type(weather_test["holidays"][0]))


<class 'numpy.int64'>
<class 'numpy.int64'>


In [ ]:
weather_train.dtypes

In [ ]:
# Encode string features into numeric features
LE = preprocessing.LabelEncoder()

train_data_all = pd.get_dummies(weather_train, columns = ['Dates', 'Category', 'DayOfWeek', 'PdDistrict',
       'Address', 'X', 'Y', 'month', 'year', 'hour', 'holidays', 'first_day', 'month_year', 'spring', 'summer',
       'fall', 'winter', 'dayparts', 'PRCP', 'TMAX', 'TMIN'])
del train_data_all["Dates"]
del train_data_all["Descript"]
del train_data_all["Resolution"]
del train_data_all["day"]
del train_data_all["doy"]
train_labels_all = np.array(train_data_all['Category'])
del train_data_all["Category"]

train_data_all["Address"] = LE.fit_transform(train_data_all["Address"])
train_data_all.reindex()

test_data_all = pd.get_dummies(weather_test, columns = ['Dates', 'DayOfWeek', 'PdDistrict',
       'Address', 'X', 'Y', 'month', 'year', 'hour', 'holidays', 'first_day', 'month_year', 'spring', 'summer',
       'fall', 'winter', 'dayparts', 'PRCP', 'TMAX', 'TMIN'])

test_data_all["Address"] = LE.fit_transform(test_data_all["Address"])
del test_data_all["Id"]
del test_data_all["day"]
del test_data_all["doy"]
del test_data_all["Dates"]
                                 
print(test_data_all.columns == train_data_all.columns)

In [109]:
print(train_data_all.head(3))
print(train_data_all.columns)

   Dates  Address           X          Y  hour  day  holidays  first_day  \
0   2248    19790 -122.425892  37.774599    23   13         0          0   
1   2248    19790 -122.425892  37.774599    23   13         0          0   
2   2248    22697 -122.424363  37.800414    23   13         0          0   

  month_year month_day         ...           year_2013  year_2014  year_2015  \
0    2015-05     05-13         ...                   0          0          1   
1    2015-05     05-13         ...                   0          0          1   
2    2015-05     05-13         ...                   0          0          1   

   dayparts_early_afternoon  dayparts_early_evening  dayparts_early_morning  \
0                         0                       0                       0   
1                         0                       0                       0   
2                         0                       0                       0   

   dayparts_late_afternoon  dayparts_late_evening  daypar

In [110]:
# Normalization
train_data_all = (train_data_all - train_data_all.mean()) / (train_data_all.std())
test_data_all = (test_data_all - train_data_all.mean())/(train_data_all.std())

KeyboardInterrupt: 

In [21]:
print(test_data_all[0:5])

    Dates  Address           X          Y  hour      holidays          PRCP  \
0  2265.0   6407.0 -122.399588  37.735051  23.0  1.134103e-15  7.976514e-14   
1  2265.0   9744.0 -122.391523  37.732432  23.0  1.134103e-15  7.976514e-14   
2  2265.0   6336.0 -122.426002  37.792212  23.0  1.134103e-15  7.976514e-14   
3  2265.0  10633.0 -122.437394  37.721412  23.0  1.134103e-15  7.976514e-14   
4  2265.0  10633.0 -122.437394  37.721412  23.0  1.134103e-15  7.976514e-14   

   TMAX  TMIN  PdDistrict_BAYVIEW         ...              year_2013  \
0  57.0  49.0        1.000000e+00         ...           3.143028e-13   
1  57.0  49.0        1.000000e+00         ...           3.143028e-13   
2  57.0  49.0       -3.280039e-14         ...           3.143028e-13   
3  57.0  49.0       -3.280039e-14         ...           3.143028e-13   
4  57.0  49.0       -3.280039e-14         ...           3.143028e-13   

      year_2014  year_2015  dayparts_early_afternoon  dayparts_early_evening  \
0 -4.016814e

In [22]:
# Shuffle data and set aside 20% as development data
train_data_all = train_data_all.values
test_data_all = test_data_all.values
n = train_data_all.shape[0]

np.random.seed(0)

shuffle = np.random.permutation(np.arange(train_data_all.shape[0]))

train_data_all = train_data_all[shuffle]
train_labels_all = train_labels_all[shuffle]

n_train = int(0.8*n)

train_data = train_data_all[:n_train,:]
train_labels = train_labels_all[:n_train]
dev_data = train_data_all[n_train:,:]
dev_labels = train_labels_all[n_train:]

In [23]:
# print shapes and some data to compare before and after csv conversion
print("train_data shape is", train_data.shape)
print("train_labels shape is", train_labels.shape)
print("dev_data shape is", dev_data.shape)
print("dev_labels shape is", dev_labels.shape)
print("train_data_all shape is", train_data_all.shape)
print("train_labels_all shape is", train_labels_all.shape)
print("test_data_all shape is", test_data_all.shape)

train_data shape is (702439, 58)
train_labels shape is (702439,)
dev_data shape is (175610, 58)
dev_labels shape is (175610,)
train_data_all shape is (878049, 58)
train_labels_all shape is (878049,)
test_data_all shape is (884262, 58)


In [24]:
# Save data as CSV files in a subdirectory

# NOTE: mkdir will make a "csv" directory in your local repo if there is not already one there.
# It will return an error if the directory already exists in your local repo
# but that will not impact how this code runs

! mkdir csv
np.savetxt("csv/train_data.csv", train_data, delimiter=",")
np.savetxt("csv/train_labels.csv", train_labels, fmt="%s", delimiter=",")
np.savetxt("csv/dev_data.csv", dev_data, delimiter=",")
np.savetxt("csv/dev_labels.csv", dev_labels, fmt="%s", delimiter=",")
np.savetxt("csv/train_data_all.csv", train_data_all, delimiter=",")
np.savetxt("csv/train_labels_all.csv", train_labels_all, fmt="%s", delimiter=",")
np.savetxt("csv/test_data_all.csv", test_data_all, delimiter=",")

mkdir: csv: File exists


In [25]:
# Zip up the CSV files

# **IMPORTANT**  This code will rewrite existing zip files in your local repo
# You will need to push it to the group repo for everyone to have the updated zip file

# Full set of training data and labels --> data.zip
zip_train_all = zipfile.ZipFile("data.zip", "w")
zip_train_all.write("csv/train_data_all.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_train_all.write("csv/train_labels_all.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_train_all.close()

# Subset of training data and labels --> data_subset.zip
zip_train_subset = zipfile.ZipFile("data_subset.zip", "w")
zip_train_subset.write("csv/train_data.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_train_subset.write("csv/train_labels.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_train_subset.close()


# Data used for testing models (test data from Kaggle and our 20% development data) --> testing.zip
zip_testing = zipfile.ZipFile("testing.zip", "w")
zip_testing.write("csv/test_data_all.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_testing.write("csv/dev_data.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_testing.write("csv/dev_labels.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_testing.close()

